In [1]:
import pandas as pd
import numpy as np
from langchain.embeddings import OpenAIEmbeddings
import os


### Read datasets

In [100]:
df_help_center = pd.read_parquet("FT_SEARCH_TERMS.parquet")
df_help_center.shape

(85410, 4)

In [24]:
df_ipc = pd.read_parquet("IPC_SEARCH_QUERY_TERMS.parquet")
df_ipc.shape

(16127, 8)

In [4]:
df_google = pd.read_parquet("PERFORMANCE_REPORT_CUSTOM.parquet")
df_google.shape

(1430248, 3)

### 1- User queries on Talend Help Center

In [11]:
df_help_center

,DATE,LANGUAGE,QUERY_TERMS,QUERY_COUNT
0,2023-05-08,en-US,management console,14
1,2023-05-08,en-US,talend cloud data catalog administration guide,9
2,2023-05-08,en-US,authentication protocols,7
3,2023-05-08,en-US,data catalog,7
4,2023-05-08,en-US,logging in to the studio,7
...,...,...,...,...
85405,2023-05-07,en-US,zip,1
85406,2023-05-07,ja-JP,management console,1
85407,2023-05-03,en-US,talend remote engine 2.12.11,1
85408,2023-05-06,en-US,precision,1


In [ ]:
# distinct queries
df_help_center["QUERY_TERMS"].unique().shape

(38492,)

In [101]:
col = df_help_center.pop('QUERY_TERMS')
df_help_center["QUERY_TERMS"] = col
df_help_center

,DATE,LANGUAGE,QUERY_COUNT,QUERY_TERMS
0,2023-05-08,en-US,14,management console
1,2023-05-08,en-US,9,talend cloud data catalog administration guide
2,2023-05-08,en-US,7,authentication protocols
3,2023-05-08,en-US,7,data catalog
4,2023-05-08,en-US,7,logging in to the studio
...,...,...,...,...
85405,2023-05-07,en-US,1,zip
85406,2023-05-07,ja-JP,1,management console
85407,2023-05-03,en-US,1,talend remote engine 2.12.11
85408,2023-05-06,en-US,1,precision


In [ ]:
# dataframe with only the distinct queries
df_distinct_queries = pd.DataFrame(df_help_center["QUERY_TERMS"].unique(), columns=['QUERY_TERMS'])
df_distinct_queries

,QUERY_TERMS
0,management console
1,talend cloud data catalog administration guide
2,authentication protocols
3,data catalog
4,logging in to the studio
...,...
38487,tmap type length precision
38488,create a job that illustrates the process of e...
38489,data base connecton
38490,selecting and configuring the database


In [16]:
# Embedding model
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=OPENAI_API_KEY)

In [104]:
# Compute the embeddings
df_distinct_queries.loc[:, "embedding"] = embeddings.embed_documents(df_distinct_queries.QUERY_TERMS.to_list())

In [125]:
# save embeddings
df_distinct_queries.to_parquet("talend_HC_unique_queries_embedding.parquet")
df_distinct_queries

,QUERY_TERMS,embedding
0,management console,"[-0.004322341693185566, -0.030163845395668177,..."
1,talend cloud data catalog administration guide,"[-0.012687883335647096, -0.0081912912401424, -..."
2,authentication protocols,"[-0.005785695361989555, -0.0050168676106133, 0..."
3,data catalog,"[-0.01183576152363635, -0.011971230765135885, ..."
4,logging in to the studio,"[-0.006270805165635671, -0.025466070624882403,..."
...,...,...
38487,tmap type length precision,"[-0.009533870630991807, 0.01000067370341025, 0..."
38488,create a job that illustrates the process of e...,"[-0.03291688505120821, -0.01156285132150887, -..."
38489,data base connecton,"[-0.027216859308402265, 0.0012151657203450083,..."
38490,selecting and configuring the database,"[0.001809233214592607, 0.004131455433434716, 0..."


In [110]:
# Merge tables based on 'QUERY_TERMS' coulumn
df_thc = pd.merge(df_help_center, df_distinct_queries, on='QUERY_TERMS', how='left')
df_thc.to_parquet("FT_SEARCH_TERMS_embeddings.parquet")
df_thc

,DATE,LANGUAGE,QUERY_COUNT,QUERY_TERMS,embedding
0,2023-05-08,en-US,14,management console,"[-0.004322341693185566, -0.030163845395668177,..."
1,2023-05-08,en-US,9,talend cloud data catalog administration guide,"[-0.012687883335647096, -0.0081912912401424, -..."
2,2023-05-08,en-US,7,authentication protocols,"[-0.005785695361989555, -0.0050168676106133, 0..."
3,2023-05-08,en-US,7,data catalog,"[-0.01183576152363635, -0.011971230765135885, ..."
4,2023-05-08,en-US,7,logging in to the studio,"[-0.006270805165635671, -0.025466070624882403,..."
...,...,...,...,...,...
85405,2023-05-07,en-US,1,zip,"[-0.012830376338424872, -0.02798850975003195, ..."
85406,2023-05-07,ja-JP,1,management console,"[-0.004322341693185566, -0.030163845395668177,..."
85407,2023-05-03,en-US,1,talend remote engine 2.12.11,"[-0.033940451557151495, -0.028766949299707138,..."
85408,2023-05-06,en-US,1,precision,"[0.0003957463227048156, -0.005160255864359095,..."


### 2- User queries on Google

In [113]:
df_google

,COUNTRY,DATE,QUERY
0,dnk,2020-03-30 17:00:00-07:00,talend sap
1,fra,2020-04-26 17:00:00-07:00,sap talend
2,fra,2020-04-05 17:00:00-07:00,tflowtoiterate talend
3,usa,2020-04-06 17:00:00-07:00,talend tftpfilelist
4,che,2020-03-30 17:00:00-07:00,talend java version
...,...,...,...
1430243,usa,2023-04-07 17:00:00-07:00,talend globalmap
1430244,deu,2023-04-07 17:00:00-07:00,docker change dns server
1430245,tun,2023-04-07 17:00:00-07:00,trowgenerator talend
1430246,tun,2023-04-07 17:00:00-07:00,tdbinput talend


In [114]:
df_google["QUERY"].unique().shape

(11474,)

In [117]:
# dataframe with only the distinct queries
df2_distinct_queries = pd.DataFrame(df_google["QUERY"].unique(), columns=['QUERY'])
df2_distinct_queries

,QUERY
0,talend sap
1,sap talend
2,tflowtoiterate talend
3,talend tftpfilelist
4,talend java version
...,...
11469,remove duplicates in ssis
11470,telecharger talend open studio for data integr...
11471,excel afficher date du jour
11472,java home windows


In [118]:
# Compute the embeddings
df2_distinct_queries.loc[:, "embedding"] = embeddings.embed_documents(df2_distinct_queries.QUERY.to_list())

In [124]:
df2_distinct_queries
# save the embeddings
df2_distinct_queries.to_parquet("google_queries_unique_embeddings.parquet")

In [120]:
# Merge tables based on 'QUERY' column
df_gg = pd.merge(df_google, df2_distinct_queries, on='QUERY', how='left')
df_gg.to_parquet("PERFORMANCE_REPORT_CUSTOM_embeddings.parquet")
df_gg

,COUNTRY,DATE,QUERY,embedding
0,dnk,2020-03-30 17:00:00-07:00,talend sap,"[-0.011422375105710116, -0.0332235988715449, 0..."
1,fra,2020-04-26 17:00:00-07:00,sap talend,"[-0.006255174100263367, -0.02995636335497445, ..."
2,fra,2020-04-05 17:00:00-07:00,tflowtoiterate talend,"[-0.04371899229381361, -0.026086970706513456, ..."
3,usa,2020-04-06 17:00:00-07:00,talend tftpfilelist,"[-0.020819874876695583, -0.02204194323452131, ..."
4,che,2020-03-30 17:00:00-07:00,talend java version,"[0.0006890799722034253, -0.036536014890475504,..."
...,...,...,...,...
1430243,usa,2023-04-07 17:00:00-07:00,talend globalmap,"[-0.027276861099226956, -0.02256860629459309, ..."
1430244,deu,2023-04-07 17:00:00-07:00,docker change dns server,"[0.0005595973455450628, -0.017678913082422275,..."
1430245,tun,2023-04-07 17:00:00-07:00,trowgenerator talend,"[-0.038557772465384825, -0.0337901942623599, -..."
1430246,tun,2023-04-07 17:00:00-07:00,tdbinput talend,"[-0.03682587858773348, -0.031629039273649434, ..."


### 3- User queries on IPC

In [18]:
df_ipc.head()

,CUSTOMER_LOCATION_DATA_COUNTRY_NAME,CUSTOMER_LOCATION_DATA_CITY_NAME,CUSTOMER_TYPE,CUSTOMER_SESSION_COUNT,INPUT_VALUES_DROPDOWN_MODULE,INPUT_VALUES_SEARCH_QUERY,SEARCH_RESULTS,REQUEST_TIME
0,United States,San Antonio,user,3,All,,[],2022-11-16T19:40:55.871380Z
1,Israel,Tel Aviv,visitor,0,All,delete account,"[\n {\n ""count"": 6,\n ""hits"": [\n ...",2022-12-25T18:35:57.034403Z
2,Canada,Toronto,visitor,0,Stitch Data Loader,,[],2023-01-05T19:59:53.091488Z
3,United States,Denver,user,281,Stitch Data Loader,convert bulk to ReST,"[\n {\n ""count"": 0,\n ""hits"": [],\n ...",2023-02-22T06:38:15.598456Z
4,Brazil,São Paulo,user,660,Talend Management Console,,[],2023-03-09T17:03:00.378696Z


In [5]:
df_ipc["INPUT_VALUES_SEARCH_QUERY"].unique().shape

(7344,)

In [9]:
# dataframe with only the distinct queries
df3_distinct_queries = pd.DataFrame(df_ipc["INPUT_VALUES_SEARCH_QUERY"].unique(), columns=['INPUT_VALUES_SEARCH_QUERY'])
df3_distinct_queries

,INPUT_VALUES_SEARCH_QUERY
0,
1,delete account
2,convert bulk to ReST
3,fetch license file
4,cannot find task version from database
...,...
7339,how to convert talend studio into talend cloud
7340,IP Whitelist
7341,full
7342,how to conect talend studio whit talend cloud


In [14]:
# delete rows that do not contain a query
df3_distinct_queries = df3_distinct_queries[df3_distinct_queries["INPUT_VALUES_SEARCH_QUERY"]!=""]
df3_distinct_queries

,INPUT_VALUES_SEARCH_QUERY
1,delete account
2,convert bulk to ReST
3,fetch license file
4,cannot find task version from database
5,character encoding
...,...
7339,how to convert talend studio into talend cloud
7340,IP Whitelist
7341,full
7342,how to conect talend studio whit talend cloud


In [ ]:
# Compute the embeddings
df3_distinct_queries.loc[:, "embedding"] = embeddings.embed_documents(df3_distinct_queries["INPUT_VALUES_SEARCH_QUERY"].to_list())

In [21]:
# save embeddings
df3_distinct_queries.to_parquet("ipc_queries_unique_embeddings.parquet")
df3_distinct_queries

,INPUT_VALUES_SEARCH_QUERY,embedding
1,delete account,"[-0.03002667820875631, -0.015312518101690856, ..."
2,convert bulk to ReST,"[-0.006475940047170754, 0.013213157973951499, ..."
3,fetch license file,"[-0.0019105491550711702, 0.001966318614425126,..."
4,cannot find task version from database,"[-0.02670016570154868, -0.008580103104015208, ..."
5,character encoding,"[-0.011962658253795, 0.0005135878070066096, 0...."
...,...,...
7339,how to convert talend studio into talend cloud,"[-0.010953764802413989, -0.022288785140119494,..."
7340,IP Whitelist,"[-0.006736527387763341, -0.019494728320854058,..."
7341,full,"[-0.004613058159430843, -0.011234858964527344,..."
7342,how to conect talend studio whit talend cloud,"[-0.02093189858074042, -0.02974008631428586, 0..."


In [ ]:
col = df_ipc.pop("INPUT_VALUES_SEARCH_QUERY")
df_help_center["QUERY_TERMS"] = col
df_help_center

In [25]:
# Merge tables based on 'INPUT_VALUES_SEARCH_QUERY' column
df_ipc = pd.merge(df_ipc, df3_distinct_queries, on='INPUT_VALUES_SEARCH_QUERY', how='left')
col = df_ipc.pop("embedding")
df_ipc.insert(6, "embedding", col)
df_ipc

,CUSTOMER_LOCATION_DATA_COUNTRY_NAME,CUSTOMER_LOCATION_DATA_CITY_NAME,CUSTOMER_TYPE,CUSTOMER_SESSION_COUNT,INPUT_VALUES_DROPDOWN_MODULE,INPUT_VALUES_SEARCH_QUERY,embedding,SEARCH_RESULTS,REQUEST_TIME
0,United States,San Antonio,user,3,All,,NaN,[],2022-11-16T19:40:55.871380Z
1,Israel,Tel Aviv,visitor,0,All,delete account,"[-0.03002667820875631, -0.015312518101690856, ...","[\n {\n ""count"": 6,\n ""hits"": [\n ...",2022-12-25T18:35:57.034403Z
2,Canada,Toronto,visitor,0,Stitch Data Loader,,NaN,[],2023-01-05T19:59:53.091488Z
3,United States,Denver,user,281,Stitch Data Loader,convert bulk to ReST,"[-0.006475940047170754, 0.013213157973951499, ...","[\n {\n ""count"": 0,\n ""hits"": [],\n ...",2023-02-22T06:38:15.598456Z
4,Brazil,São Paulo,user,660,Talend Management Console,,NaN,[],2023-03-09T17:03:00.378696Z
...,...,...,...,...,...,...,...,...,...
16122,India,Chennai,user,62,Talend Management Console,how to convert talend studio into talend cloud,"[-0.010953764802413989, -0.022288785140119494,...","[\n {\n ""count"": 6,\n ""hits"": [\n ...",2023-03-13T09:11:52.992262Z
16123,India,Mumbai,contact,0,Stitch Data Loader,IP Whitelist,"[-0.006736527387763341, -0.019494728320854058,...","[\n {\n ""count"": 0,\n ""hits"": [],\n ...",2023-03-28T09:44:01.387815Z
16124,Colombia,Medellín,user,5,Stitch Data Loader,full,"[-0.004613058159430843, -0.011234858964527344,...","[\n {\n ""count"": 0,\n ""hits"": [],\n ...",2023-04-18T02:58:36.198569Z
16125,United States,Reston,user,3,Talend Studio,how to conect talend studio whit talend cloud,"[-0.02093189858074042, -0.02974008631428586, 0...","[\n {\n ""count"": 6,\n ""hits"": [\n ...",2023-04-17T18:46:47.493998Z


In [26]:
df_ipc.to_parquet("IPC_SEARCH_QUERY_TERMS_embeddings.parquet")